# INVALID_CONCURRENT_GRAPH_UPDATE

A LangGraph [`StateGraph`](https://langchain-ai.github.io/langgraphjs/reference/classes/langgraph.StateGraph.html) received concurrent updates to its state from multiple nodes to a state property that doesn't
support it.

One way this can occur is if you are using a [fanout](https://langchain-ai.github.io/langgraphjs/how-tos/map-reduce/)
or other parallel execution in your graph and you have defined a state with a value like this:

```ts
const StateAnnotation = Annotation.Root({
  someKey: Annotation<string>,
});

const graph = new StateGraph(StateAnnotation)
  .addNode(...)
  ...
  .compile();
```

If a node in the above graph returns `{ someKey: "someStringValue" }`, this will overwrite the state value for `someKey` with `"someStringValue"`.
However, if multiple nodes in e.g. a fanout within a single step return values for `"someKey"`, the graph will throw this error because
there is uncertainty around how to update the internal state.

To get around this, you can define a reducer that combines multiple values:

```ts
const StateAnnotation = Annotation.Root({
  someKey: Annotation<string[]>({
    default: () => [],
    reducer: (a, b) => a.concat(b),
  }),
});
```

This will allow you to define logic that handles the same key returned from multiple nodes executed in parallel.

## Troubleshooting

The following may help resolve this error:

- If your graph executes nodes in parallel, make sure you have defined relevant state keys with a reducer.
